In [1]:
import numpy as np
from gym_env.envs import ground_env as ge
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3 import DQN, A2C, PPO
import gym
import os
import glob
import random
import matplotlib.pyplot as plt
import plotly.graph_objects as go

/Users/kowshik/projects/personal/reinforcement_learning_optimal_policy/playenv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
grid_shape = 1000
rewards_dict = {'cosine_distance_scale': 10000, 'penalty_for_one_step': 0,
 'penalty_for_one_step_when_ball_is_here': 1, 'reward_for_catch':100, 'time_between_catch_next_ball':100,
  'game_end_total':1000,'game_end_pickup_time':10000}
length_of_test = 10*1e3
  
reward_params = list(rewards_dict.values())
env = ge.playground_env(grid_shape,'center','radial',reward_params =  reward_params)
model = DQN("MlpPolicy", env, verbose=0, tensorboard_log="sac",learning_rate=0.001)
model.learn(total_timesteps=5*1e5, log_interval=4,)
model.save("models/station_policy")


In [9]:
model =  DQN.load('models/station_policy')
done = False
reward_params_test = reward_params
reward_params_test[-1] = length_of_test
env = ge.playground_env(grid_shape,'center','radial',reward_params = reward_params)
obs = env.reset()
action_profile = []
reward_profile = []
frames_ = []
X_station = []
Y_station = []
c = 0
while not done:
    c = c +1 
    act = model.predict(obs)
    obs_, reward, done, info = env.step(act[0])
    obs = obs_
    action_profile.append([act[0]])
    reward_profile.append(reward)
    if env.PG.generated:
        frames_.append(go.Frame(data=[go.Scatter(x=[env.PG.pos[0], env.PG.ran_pos[0]], y=[env.PG.pos[1], env.PG.ran_pos[1]])]))
    else:
        frames_.append(go.Frame(data=[go.Scatter(x=[env.PG.pos[0]], y=[env.PG.pos[1]])]))
        X_station.append(env.PG.pos[0])
        Y_station.append(env.PG.pos[1])

In [10]:
from turtle import color
import plotly.graph_objects as go
fig = go.Figure(
    data=[go.Scatter(mode='markers',marker=dict(size=[15, 40], color = ['green','red']), text=['🚶','⚽️'], marker_symbol= ['star-triangle-up','circle'])],
    layout=go.Layout(
        xaxis=dict(range=[0, grid_shape], autorange=False),
        yaxis=dict(range=[0, grid_shape], autorange=False),
        title="Start Title",
        updatemenus=[dict(buttons = [dict(
                                        args = [None, {"frame": {"duration": 5, 
                                                                "redraw": False},
                                                        "fromcurrent": True, 
                                                        "transition": {"duration": 0}}],
                                        label = "Play",
                                        method = "animate")],
                                type='buttons',
                                showactive=False)]
    ),
    frames=frames_
)
fig.update_layout(width= 600, height= 600)
fig.show()